In [ ]:
# default_exp cp

# Competitive Programming Library

> Things to be used by copy & paste. Also useful for other situtations as well!

(Not callable from atomlib).

In [ ]:
#hide
from atomlib.utils import timing

## Prime Number Problems

### Check for primes

Find wether a number is prime or not

In [ ]:
def is_prime(n: int) -> bool:
    # Corner cases 
    if (n <= 1) : 
        return False
    if (n <= 3) : 
        return True
  
    # This is checked so that we can skip  
    # middle five numbers in below loop 
    if (n % 2 == 0 or n % 3 == 0) : 
        return False
  
    i = 5
    while(i * i <= n) : 
        if (n % i == 0 or n % (i + 2) == 0) : 
            return False
        i = i + 6
  
    return True

In [ ]:
%timeit is_prime(10**9+7)

944 µs ± 30.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Sieve of Eratosthenes

Gives all prime numbers less than or equal to n.

In [ ]:

def sieve_of_eratosthenes(n):
    prime = [True for i in range(n+1)]
    p = 2
    while p**2 <= n:
        if prime[p] == True:
            
            for i in range(p**2, n+1, p):
                prime[i] = False
        p += 1
        
    return [p for p in range(2, n) if prime[p]]

In [ ]:
%timeit sieve_of_eratosthenes(10**6)

151 ms ± 6.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Legendre’s formula

Given an integer $n$ and a prime number $p$, find the largest $x$ such that $p^x$ divides $n!$

In [ ]:
def largest_power(n, p): 
    x = 0
    while n: 
        n = n // p 
        x += n 
    return x 

%timeit largest_power(10**18, 27)

1.34 µs ± 41.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Combinatrics

Resources: 

- [Combinatorices, Dr. Phillip M. Feldman](https://phillipmfeldman.org/Python/combinatorics.html)

### Multinomial Coefficient


A multinomial coefficient is often written in the form

$$\binom{n}{k_1,k_2,\ldots,k_m}$$

where $k_1+k_2+\cdots+k_m = n.$ Because of that last equation, the n is redundant. Binomial coefficients are technically multinomial coefficients as well, but instead of writing $\binom{n}{k_1,k_2}$ where $k1+k2=n$, we usually write either $\binom{n}{k_1}$ or $\binom{n}{k_2}$. These are all mean exactly the same thing:

$$\binom{k_1+k_2}{k_1,k_2}=\binom{k_1+k_2}{k_1}=\binom{k_1+k_2}{k_2} = \frac{(k_1+k_2)!}{k_1!\,k_2!}.$$

A multinomial coefficient can arise in a counting problem as follows:

- You have **$m$ containers**, with sizes $k_1,k_2,…,k_m$ respectively.
- You have $n=k_1+k_2+⋯+k_m$ distinguishable objects, which are just enough to fill every container when drawing from the n objects **without replacement**.
- The order of the containers matters.
- The order of the objects within each container does not matter.

In [ ]:
#export
def multinomial_coefficients(K):
    """
    Returns the multinomial coefficients
    
    - `K` - A list of $k_1, k_2...$
    """
    res, i = 1, sum(K)
    i0 = K.index(max(K))
    for a in K[:i0] + K[i0+1:]:
        for j in range(1,a+1):
            res *= i
            res //= j
            i -= 1
    return res

def multinomial_combinations(items):
    return set(permutations(items))

Examples:

- **How many words can you make from the letters in *coffee*?**

The word *coffee* contains the letters C×1, O×1, F×2, E×2. Therefore the permutations will be,

$$\binom{6}{1, 1, 2, 2} = \frac{6!}{1!1!2!2!} = 180$$

- **How many ways to divide a class of 15 students into 3 groups, A, B and C?**

$$\binom{15}{3, 3, 3, 3, 3} = \frac{15!}{(3!)^5} = 168168000$$

In [ ]:
def multinomial_coefficients(K):
    """
    Computes the multinomial coefficient
    K = [k1, k2, k3, k4...]
    src: https://stackoverflow.com/a/46378809/5034709
    """
    res, i = 1, sum(K)
    i0 = K.index(max(K))
    for a in K[:i0] + K[i0+1:]:
        for j in range(1,a+1):
            res *= i
            res //= j
            i -= 1
    return res

K = [3,3,3,3,3]
print(multinomial_coefficients(K))

K = ['MISSISSIPPI'.count(c) for c in set('MISSISSIPPI')]
print(multinomial_coefficients(K))

168168000
34650


Very slow method of computing multinomial combinations using permutations and sets

In [ ]:
from itertools import permutations

def multinomial_combinations_slow(items):
    return set(permutations(items))

items = [c for c in 'MISSISSIPPI']
with timing():
    combs = multinomial_combinations_slow(items)
    print(next(iter(combs)))
    print(len(combs))

('S', 'S', 'S', 'P', 'I', 'I', 'P', 'M', 'S', 'I', 'I')
34650
Time: 3.9913318157196045


Below is a faster dynamic programming solution.

In [ ]:
from itertools import combinations

from itertools import chain,repeat,islice,count
from collections import Counter

def combinations_without_repetition(r, iterable=None, values=None, counts=None):
    if iterable:
        values, counts = zip(*Counter(iterable).items())

    f = lambda i,c: chain.from_iterable(map(repeat, i, c))
    n = len(counts)
    indices = list(islice(f(count(),counts), r))
    if len(indices) < r:
        return
    while True:
        yield tuple(values[i] for i in indices)
        for i,j in zip(reversed(range(r)), f(reversed(range(n)), reversed(counts))):
            if indices[i] != j:
                break
        else:
            return
        j = indices[i]+1
        for i,j in zip(range(i,r), f(count(j), islice(counts, j, None))):
            indices[i] = j

with timing():
    combs = combinations_without_repetition(len(items),items)
    print(list(combs))
#     print(next(iter(combs)))
#     print(len(combs))

[('M', 'I', 'I', 'I', 'I', 'S', 'S', 'S', 'S', 'P', 'P')]
Time: 0.0003483295440673828


In [ ]:
    if len(K) == 1:
        for c in combinations(items, K[0]):
            yield (c,)
    else:
        for c_first in combinations(items, K[0]):
            items_remaining = []
            for c_other in m_way_ordered_combinations(items_remaining, K[1:]):
                yield (c_first,) + c_other


In [ ]:
import numpy, itertools

def multiset_combinations(n, k, max_power=None):
    """returns a list (2d numpy array) of all length k sequences of 
    non-negative integers n1, ..., nk such that n1 + ... + nk = n."""
    bar_placements = itertools.combinations(range(1, n+k), k-1)
    tmp = [(0,) + x + (n+k,) for x in bar_placements]
    sequences =  numpy.diff(tmp) - 1
    if max_power:
        return sequences[numpy.where((sequences<=max_power).all(axis=1))][::-1]
    else:
        return sequences[::-1]

K = 3
multinomial_combinations(5, K)

array([[5, 0, 0],
       [4, 1, 0],
       [4, 0, 1],
       [3, 2, 0],
       [3, 1, 1],
       [3, 0, 2],
       [2, 3, 0],
       [2, 2, 1],
       [2, 1, 2],
       [2, 0, 3],
       [1, 4, 0],
       [1, 3, 1],
       [1, 2, 2],
       [1, 1, 3],
       [1, 0, 4],
       [0, 5, 0],
       [0, 4, 1],
       [0, 3, 2],
       [0, 2, 3],
       [0, 1, 4],
       [0, 0, 5]])

## Multiset Coefficient

A multiset coefficient is often written in the form

$$\binom{n+k-1}{k}$$

A multiset coefficient can arise as follows:

- You have n distinguishable objects.
- You have a **single container** able to hold k objects.
- You draw objects from the n objects with replacement, and put the copies of those objects into the container.
- You continue putting copies of objects in the container until it is full.
- The order of objects in the container does not matter.

A multiset coefficient can alternatively arise this way:

- You have k objects that are indistinguishable or that you choose not to distinguish.
- You have n containers, each with effectively infinite capacity (they never get full).
- You draw the objects without replacement and put them in the containers.
- The order of the containers matters.
- The order of objects in a container cannot matter, because by assumption you cannot tell which object is which, only how many are in each container.


<hr>

## Memoization

### Factorial

Calculates all the factorials up to n, $O(n)$.

Max n is $10^4$. If theres a large prime look at the `factorial_mod()` solution.

In [ ]:
def init_factorial(n):
    facs = [1]
    for i in range(1, n+1):
        facs.append(facs[i-1] * i)
    return facs

facs = init_factorial(10**4)

<hr>

## Modulo 10**9 + 7 Problems

### Binomial Modulo

Fast way to solve $nCr \ \ \% \ \ MOD $ when MOD is prime.

In [ ]:
def binomial_mod(n, k, p):
    numerator = 1
    for i in range(k):
        numerator = (numerator * (n-i)) % p

    denominator = 1
    for i in range(1, k+1):
        denominator = (denominator * i) % p

    return (numerator * pow(denominator, p-2, p)) % p

MOD = 10**9+7
%timeit binomial_mod(10**9, 10**6, MOD)

285 ms ± 5.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Factorial Modulo

Fast way to solve $n!\ \ \% \ \ MOD$ when MOD is prime

In [ ]:
def factorial_mod(n, p) : 
    ans = 1
    for i in range(1, n+1):
        ans = ans * i % p
    return ans

MOD = 10**9+7
%timeit factorial_mod(10**6, MOD)

125 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
